In [2]:
from google.colab import files
uploaded = files.upload()


Saving download-removebg-preview.png to download-removebg-preview.png
Saving coffee-removebg-preview.png to coffee-removebg-preview.png


In [3]:
# Install the necessary libraries
!pip install diffusers[torch]
!pip install torch torchvision torchaudio
!pip install opencv-python-headless  # For OpenCV to create the video


In [5]:
import torch
from PIL import Image
from diffusers import StableDiffusionPipeline
import cv2
import os

# Create output directory for frames
output_dir = './frames'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the static background (to ensure consistency)
def load_static_background(background_path):
    try:
        background = Image.open(background_path).convert("RGBA")
        return background
    except Exception as e:
        print(f"Error loading background image: {e}")
        return None

# Dynamically resize the object for zoom-out effect
def dynamic_resize_zoom_out(object_image, background, scale_factor):
    bg_width, bg_height = background.size
    # Resize the object image gradually based on the scale factor
    object_width = int(bg_width * scale_factor)
    object_height = int(object_image.height * (object_width / object_image.width))  # Maintain aspect ratio
    object_image = object_image.resize((object_width, object_height), Image.LANCZOS)
    return object_image

# Place the object dynamically
def place_object(background, object_image, position="center"):
    bg_width, bg_height = background.size
    obj_width, obj_height = object_image.size

    # Place the object at the center
    x = (bg_width - obj_width) // 2
    y = (bg_height - obj_height) // 2

    # Paste the object onto the background using the alpha channel for transparency
    background.paste(object_image, (x, y), object_image)
    return background

# Generate multiple frames with the zoom-out effect
def generate_frames(image_path, background_path, num_frames=20):
    # Load the coffee cup and background image
    object_image = Image.open(image_path).convert("RGBA")
    background = load_static_background(background_path)

    # Generate frames
    for i in range(num_frames):
        # Scale factor decreases over time to simulate zoom out
        scale_factor = 0.3 + i * 0.02  # Start at 30% of the background width and zoom out gradually

        # Resize the object
        resized_object = dynamic_resize_zoom_out(object_image, background, scale_factor)

        # Place the object
        final_image = place_object(background.copy(), resized_object)

        # Save the frame
        frame_path = os.path.join(output_dir, f"frame_{i:03d}.png")
        final_image.save(frame_path)
        print(f"Frame {i+1}/{num_frames} saved.")

# Generate the frames with zoom-out effect
image_path = 'coffee-removebg-preview.png'  # Path to the coffee cup image
background_path = 'download-removebg-preview.png'  # Path to the static background
generate_frames(image_path, background_path)


Frame 1/20 saved.
Frame 2/20 saved.
Frame 3/20 saved.
Frame 4/20 saved.
Frame 5/20 saved.
Frame 6/20 saved.
Frame 7/20 saved.
Frame 8/20 saved.
Frame 9/20 saved.
Frame 10/20 saved.
Frame 11/20 saved.
Frame 12/20 saved.
Frame 13/20 saved.
Frame 14/20 saved.
Frame 15/20 saved.
Frame 16/20 saved.
Frame 17/20 saved.
Frame 18/20 saved.
Frame 19/20 saved.
Frame 20/20 saved.


In [6]:
import cv2
import os

def create_video_from_frames(output_dir, video_name="output_video.mp4", fps=10):
    # Get all frame files in the output directory
    frame_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.png')])

    # Load the first frame to get video dimensions
    first_frame = cv2.imread(os.path.join(output_dir, frame_files[0]))
    height, width, layers = first_frame.shape

    # Create the video writer
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    # Write each frame into the video
    for frame_file in frame_files:
        frame = cv2.imread(os.path.join(output_dir, frame_file))
        video.write(frame)

    # Release the video writer
    video.release()
    print(f"Video saved as {video_name}")

# Create the video from frames
create_video_from_frames(output_dir, video_name="output_video.mp4")


Video saved as output_video.mp4


In [7]:
from google.colab import files
files.download("output_video.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>